## Анализ результатов A/B-тестирования

Теперь вам нужно проанализировать другие данные. Представьте, что к вам обратились представители интернет-магазина BitMotion Kit, в котором продаются геймифицированные товары для тех, кто ведёт здоровый образ жизни. У него есть своя целевая аудитория, даже появились хиты продаж: эспандер со счётчиком и напоминанием, так и подстольный велотренажёр с Bluetooth.

В будущем компания хочет расширить ассортимент товаров. Но перед этим нужно решить одну проблему. Интерфейс онлайн-магазина слишком сложен для пользователей — об этом говорят отзывы.

Чтобы привлечь новых клиентов и увеличить число продаж, владельцы магазина разработали новую версию сайта и протестировали его на части пользователей. По задумке, это решение доказуемо повысит количество пользователей, которые совершат покупку.

Ваша задача — провести оценку результатов A/B-теста. В вашем распоряжении:

* данные о действиях пользователей и распределении их на группы,

* техническое задание.

Оцените корректность проведения теста и проанализируйте его результаты.

### Техническое задание проведённого теста

Предыдущий аналитик проверял полное обновление дизайна сайта. Гипотеза заключается в следующем: упрощение интерфейса приведёт к тому, что в течение семи дней после регистрации в системе конверсия зарегистрированных пользователей в покупателей увеличится как минимум на три процентных пункта.

Параметры теста:

- название теста: `interface_eu_test`;
- группы: А (контрольная), B (новый интерфейс);
- дата набора новых пользователей: с 2020-12-01 по 2020-12-21 включительно;
- дата прохождения теста: с 2020-12-08 по 2020-12-29 включительно.

Вам нужно:

* загрузить данные теста;

* проверить корректность его проведения;

* проанализировать полученные результаты.

### Данные

- `https://code.s3.yandex.net/datasets/ab_test_participants.csv` — таблица участников тестов.
  Структура файла:

  - `user_id` — идентификатор пользователя;
  - `group` — группа пользователя;
  - `ab_test` — название теста;
  - `device` — устройство, с которого происходила регистрация.

- `https://code.s3.yandex.net/datasets/ab_test_events.zip` — архив с одним `csv`-файлом, в котором собраны события 2020 года;
  Структура файла:

  - `user_id` — идентификатор пользователя;
  - `event_dt` — дата и время события;
  - `event_name` — тип события;
  - `details` — дополнительные данные о событии.


### Как выполнить задание:

1. Опишите цели исследования.

**Цель исследования** - выяснить, повлиет ли обновленный интерфейс сайта на увеличение конверсии пользователей в покупателей.

Для этого будет проведен Z-тест пропорций на двух группах пользователей.

2. Загрузите данные, оцените их целостность.

In [1]:
import pandas as pd

URL = 'https://code.s3.yandex.net/datasets/'
participants = pd.read_csv(URL + 'ab_test_participants.csv')
events = pd.read_csv(URL + 'ab_test_events.zip', parse_dates=['event_dt'])

/opt/conda/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [2]:
display(participants.info())
display(events.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14525 entries, 0 to 14524
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  14525 non-null  object
 1   group    14525 non-null  object
 2   ab_test  14525 non-null  object
 3   device   14525 non-null  object
dtypes: object(4)
memory usage: 454.0+ KB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 787286 entries, 0 to 787285
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   user_id     787286 non-null  object        
 1   event_dt    787286 non-null  datetime64[ns]
 2   event_name  787286 non-null  object        
 3   details     249022 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 24.0+ MB


None

В таблице `participants` содержится **14 525** данных без пропусков. В таблице `events` содержится **787 286** данных, значительные пропуски присутствуют в столбце `details`.

In [3]:
display(participants.head())
display(events.head())

,user_id,group,ab_test,device
0,0002CE61FF2C4011,B,interface_eu_test,Mac
1,001064FEAAB631A1,B,recommender_system_test,Android
2,001064FEAAB631A1,A,interface_eu_test,Android
3,0010A1C096941592,A,recommender_system_test,Android
4,001E72F50D1C48FA,A,interface_eu_test,Mac


,user_id,event_dt,event_name,details
0,GLOBAL,2020-12-01 00:00:00,End of Black Friday Ads Campaign,ZONE_CODE15
1,CCBE9E7E99F94A08,2020-12-01 00:00:11,registration,0.0
2,GLOBAL,2020-12-01 00:00:25,product_page,NaN
3,CCBE9E7E99F94A08,2020-12-01 00:00:33,login,NaN
4,CCBE9E7E99F94A08,2020-12-01 00:00:52,product_page,NaN


3. По таблице `ab_test_participants` оцените корректность проведения теста:

   3\.1 Выделите пользователей, участвующих в тесте, и проверьте:

   - соответствие требованиям технического задания,

   - равномерность распределения пользователей по группам теста,

   - отсутствие пересечений с конкурирующим тестом (нет пользователей, участвующих одновременно в двух тестовых группах).

In [4]:
ab_test_participants = participants[participants['ab_test'] == 'interface_eu_test']

In [5]:
#Проверяем наличие тестовых групп
ab_test_participants['group'].value_counts()

B    5467
A    5383
Name: group, dtype: int64

В данных две группы (A и B) с примерно одинаковым количеством наблюдений

In [6]:
display(events[events['user_id'].isin(ab_test_participants['user_id'])]['event_dt'].min())
display(events[events['user_id'].isin(ab_test_participants['user_id'])]['event_dt'].max())

Timestamp('2020-12-06 14:10:01')

Timestamp('2020-12-30 18:34:08')

В данных отсутсвует период **2020-12-01 по 2020-12-05** и присутсвует лишний день **2020-12-30**. Кроме этого период сбора данных совпадает с техническим заданием.

Параметры теста совпадают с техническим заданием, за исключением несоответствия периодов сбора данных.

In [7]:
test_groups = participants.groupby('user_id')['ab_test'].nunique().reset_index()
test_groups_more_than_one = test_groups[test_groups['ab_test'] > 1]
display(test_groups_more_than_one.shape[0])

887

В выборке содержится 887 пользователей, принимавших участие в более чем двух тестах

In [8]:
participants[participants['user_id'].isin(test_groups_more_than_one['user_id'])]['ab_test'].unique()

array(['recommender_system_test', 'interface_eu_test'], dtype=object)

Пересечения в том числе присутствуют для теста `interface_eu_test`, исследуемого в работе. Проверим также наличие пересечения контрольной и тестовых групп:

In [9]:
test_groups_A_B = ab_test_participants.groupby('user_id')['group'].nunique().reset_index()
test_groups_A_B_more_than_one = test_groups_A_B[test_groups_A_B['group'] > 1]
display(test_groups_A_B_more_than_one.shape[0])

0

В данных нет пересечений по тестовым группам A и B, удалим из данных пользователей, участвовавших сразу в нескольких экспериментах:

In [10]:
ab_test_participants_cleaned = ab_test_participants[~ab_test_participants['user_id'].isin(test_groups_more_than_one['user_id'])]

Оцените достаточность выборки для получения статистически значимых результатов A/B-теста. Заданные параметры: 

- базовый показатель конверсии — 30%,

- мощность теста — 80%,

- достоверность теста — 95%.

In [11]:
ab_test_participants_events = ab_test_participants_cleaned.merge(events, on = 'user_id', how = 'inner')

In [12]:
A_group = ab_test_participants_events[ab_test_participants_events['group'] == 'A']
B_group = ab_test_participants_events[ab_test_participants_events['group'] == 'B']

# Количество покупок
A_purchase = A_group[A_group['event_name'] == 'purchase'].shape[0]
B_purchase = B_group[B_group['event_name'] == 'purchase'].shape[0]

# Доля успешных сессий
A_purchase_share = A_purchase / A_group.shape[0]
B_purchase_share = B_purchase / B_group.shape[0]

Проверим достаточность выборок:

In [13]:
if (A_purchase_share*A_group.shape[0] > 10)and (B_purchase_share*B_group.shape[0] > 10):
    print('Предпосылка о достаточном количестве данных выполняется!')
else:
    print('Предпосылка о достаточном количестве данных НЕ выполняется!')

Предпосылка о достаточном количестве данных выполняется!


3\.2 Проанализируйте данные о пользовательской активности по таблице `ab_test_events`:

- оставьте только события, связанные с участвующими в изучаемом тесте пользователями;

In [14]:
ab_test_events = events[events['user_id'].isin(ab_test_participants_cleaned['user_id'])]

In [15]:
ab_test_events.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73815 entries, 64672 to 780371
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   user_id     73815 non-null  object        
 1   event_dt    73815 non-null  datetime64[ns]
 2   event_name  73815 non-null  object        
 3   details     19450 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 2.8+ MB


In [16]:
# Проверим наличие дубликатов
ab_test_events.duplicated().sum()

5741

In [17]:
# Отчистим данные от дубликатов
ab_test_events_cleaned = ab_test_events.drop_duplicates(keep = 'first', inplace=False)

In [18]:
ab_test_events_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68074 entries, 64672 to 780371
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   user_id     68074 non-null  object        
 1   event_dt    68074 non-null  datetime64[ns]
 2   event_name  68074 non-null  object        
 3   details     18864 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 2.6+ MB


- определите горизонт анализа: рассчитайте время (лайфтайм) совершения события пользователем после регистрации и оставьте только те события, которые были выполнены в течение первых семи дней с момента регистрации;

In [19]:
# Выбираем даты регистрации по техническому заданию
registrations = ab_test_events_cleaned[(ab_test_events_cleaned['event_name'] == 'registration') & 
                                       (ab_test_events_cleaned['event_dt'] >= '2020-12-01') & 
                                       (ab_test_events_cleaned['event_dt'] <= '2020-12-21')][['user_id', 'event_dt']]
registrations.rename(columns={'event_dt': 'registration_date'}, inplace=True)

# Добавления даты регистрации к данным
ab_test_events_cleaned = ab_test_events_cleaned.merge(registrations, on='user_id', how='left')

# Оставляем данные в течение 7 дней после момента регистрации, саму регистрацию в данные не включаем
ab_test_events_filtered = ab_test_events_cleaned[
             (ab_test_events_cleaned['event_dt'] <= ab_test_events_cleaned['registration_date'] + pd.Timedelta(days=7)) &
             (ab_test_events_cleaned['event_name'] != 'registration')]
display(ab_test_events_filtered.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39242 entries, 15 to 65190
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   user_id            39242 non-null  object        
 1   event_dt           39242 non-null  datetime64[ns]
 2   event_name         39242 non-null  object        
 3   details            4515 non-null   object        
 4   registration_date  39242 non-null  datetime64[ns]
dtypes: datetime64[ns](2), object(3)
memory usage: 1.8+ MB


None

- рассчитайте для каждой группы количество посетителей, сделавших покупку, и общее количество посетителей.

In [20]:
# Присоединяем информацию о тестовых группах
ab_test_events_final = ab_test_events_filtered.merge(ab_test_participants_cleaned[['user_id','group']], 
                                                     on = 'user_id', how = 'inner')
ab_test_events_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39242 entries, 0 to 39241
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   user_id            39242 non-null  object        
 1   event_dt           39242 non-null  datetime64[ns]
 2   event_name         39242 non-null  object        
 3   details            4515 non-null   object        
 4   registration_date  39242 non-null  datetime64[ns]
 5   group              39242 non-null  object        
dtypes: datetime64[ns](2), object(4)
memory usage: 2.1+ MB


In [21]:
A_events = ab_test_events_final[ab_test_events_final['group'] == 'A']
B_events = ab_test_events_final[ab_test_events_final['group'] == 'B']

# Количество покупок
A_events_purchase = A_events[A_events['event_name'] == 'purchase'].shape[0]
B_events_purchase = B_events[B_events['event_name'] == 'purchase'].shape[0]

# Общее количество данных
A_events_all = A_events.shape[0]
B_events_all = B_events.shape[0]

# Конверсия в покупку
A_events_conv = round(A_events_purchase / A_events_all, 4)
B_events_conv = round(B_events_purchase / B_events_all, 4)

print(f'Общее количество данных в выборке А: {A_events_all}, в выборке B: {B_events_all}')
print(f'Количество покупок в выборке А: {A_events_purchase}, в выборке B: {B_events_purchase}')

Общее количество данных в выборке А: 19120, в выборке B: 20122
Количество покупок в выборке А: 1982, в выборке B: 2533


- сделайте предварительный общий вывод об изменении пользовательской активности в тестовой группе по сравнению с контрольной.

In [22]:
print(f'Конверсия в покупку в выборке А: {A_events_conv}, в выборке B: {B_events_conv}')
print(f'Разница в конверсии между тестовой и контрольной выборками составила: {round(B_events_conv - A_events_conv, 2)}')

Конверсия в покупку в выборке А: 0.1037, в выборке B: 0.1259
Разница в конверсии между тестовой и контрольной выборками составила: 0.02


По собранным данным конверсия в тестовой выборке оказалось **больше** чем в контрольной **на 2 процентных пункта**

4. Проведите оценку результатов A/B-тестирования:

- Проверьте изменение конверсии подходящим статистическим тестом, учитывая все этапы проверки гипотез.

In [23]:
from statsmodels.stats.proportion import proportions_ztest

In [24]:
alpha = 0.05

stat_ztest, p_value_ztest = proportions_ztest(
[A_events_purchase, B_events_purchase],
[A_events_all, B_events_all],
alternative='smaller'
)

if p_value_ztest > alpha:
    print(f'pvalue={p_value_ztest} > {alpha}')
    print('Не получилось отвергнуть Нулевую гипотезу')
else:
    print(f'pvalue={p_value_ztest} < {alpha}')
    print('Нулевая гипотеза не находит подтверждения')

pvalue=2.686477338038249e-12 < 0.05
Нулевая гипотеза не находит подтверждения


- Опишите выводы по проведённой оценке результатов A/B-тестирования. Что можно сказать про результаты A/B-тестирования? Был ли достигнут ожидаемый эффект в изменении конверсии?

В результате работы был проведен Z-тест пропорций с уровнем значимости 5%. P-value получился меньше заданного уровня значимости, поэтому мы можем говорить о статистически значимом изменении конверсии пользователей в покупателей после обновления интерфейса приложения.

Однако предположение о том, что изменение интерфейса увеличит конверсию не менее чем на 3 процентных пункта не подтвердилось, так как разница средних между двумя выборками составила 2 процентных пункта.